<a href="https://colab.research.google.com/github/ov-nastia/Machine_learning_Ovchinnik/blob/main/lab_7_2_Ovchinnik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Побудуйте власну рекомендаційну систему (приклад є в документації). Оцініть її. Отримайте рекомендацію для певного користувача.
!pip install scikit-Surprise
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import train_test_split
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-Surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357264 sha256=c0ce6210bba569de620d1d44ca0088de1e63e319fc702bcc7f73ca85b8b63acf
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-Surprise


In [5]:
class MyOwnAlgorithm(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        self.user_means = {}
        for uid, ratings in trainset.ur.items():
            self.user_means[uid] = np.mean([rating for (_, rating) in ratings])
        return self

    def estimate(self, uid, iid):
        if uid in self.user_means:
            return self.user_means[uid]
        else:
            return self.trainset.global_mean

In [6]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

model = MyOwnAlgorithm()
model.fit(trainset)

In [7]:
from surprise import accuracy
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.0531
MAE:  0.8441


In [8]:
user_id = str(244)

all_items = set(trainset.all_items())
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Топ-10 фільмів, рекомендованих для користувача:
Фільм 880 з прогнозованим рейтингом 3.70
Фільм 474 з прогнозованим рейтингом 3.70
Фільм 331 з прогнозованим рейтингом 3.70
Фільм 405 з прогнозованим рейтингом 3.70
Фільм 610 з прогнозованим рейтингом 3.70
Фільм 73 з прогнозованим рейтингом 3.70
Фільм 50 з прогнозованим рейтингом 3.70
Фільм 202 з прогнозованим рейтингом 3.70
Фільм 928 з прогнозованим рейтингом 3.70
Фільм 687 з прогнозованим рейтингом 3.70
